In [222]:
from bs4 import BeautifulSoup
import urllib2
import csv
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import urllib
import pandas as pd
from random import randint
import json

In [223]:
movie = pd.read_csv('moviedata.csv')
movies_data = pd.read_csv('Movies.csv')
user_rating = pd.read_csv('User_rating.csv')

## Input

In [224]:
for i in range(0,5):
    movieid = randint(1,100)
    a = movies_data.loc[movies_data['MovieID'] == movieid,['Name']].iloc[0]['Name']
    rated = input("Enter rating for: " + a)
    user_rating = user_rating.append({'UserID': 1000, 'MovieID': movieid, 'Rated': rated}, ignore_index=True)

Enter rating for: Unbreakable Kimmy Schmidt3
Enter rating for: Once Upon a Time in Hollywood1
Enter rating for: Vikings2
Enter rating for: Gotham4
Enter rating for: Once Upon a Time in Hollywood5


## Item - Item

In [225]:
max = user_rating.loc[user_rating['UserID'] == 1000, ['Rated']]
max2 = max['Rated'].astype(float).idxmax()
max_movie_id = user_rating.iloc[max2]['MovieID']
max_movie_name1 = movies_data.loc[movies_data['MovieID'] == max_movie_id, ['Name']]
max_movie_name = max_movie_name1.iloc[0]['Name']
movie_data = pd.merge(movies_data,user_rating, on='MovieID')
user_movie_rating = movie_data.pivot_table(index='UserID', columns='Name', values='Rated',aggfunc='first')
ratings = user_movie_rating[str(max_movie_name)]  
movies_like = user_movie_rating.corrwith(ratings,method='pearson')
corr= pd.DataFrame(movies_like, columns=['Correlation'])
corr.dropna(inplace=True)
corr = corr.sort_values('Correlation', ascending=False).head(6)
print "Because u watched the movie : " + max_movie_name
recommends = list(corr.index)
recommends.pop(0)
for s in recommends:
    print s

Because u watched the movie : Once Upon a Time in Hollywood
Polar
Deadpool 2
Roswell, New Mexico
Avengers: Infinity War
Bohemian Rhapsody


# User - User

In [226]:
movie_user_rating = movie_data.pivot_table(index= 'Name',columns = 'UserID',values='Rated',aggfunc= 'first')
usern_rating = movie_user_rating[1000]
user_like_user = movie_user_rating.corrwith(usern_rating,method='pearson')
corr_user = pd.DataFrame(user_like_user, columns=['Correlation'])  
corr_user.dropna(inplace=True)  
corr_user = corr_user.sort_values('Correlation', ascending=False).head(10) 
similar_user = list(corr_user.index)[1]
similar_user_rating = movie_user_rating[similar_user]
usern_rating.dropna(inplace=True)
similar_user_rating.dropna(inplace=True)
similar_user_rating = similar_user_rating.sort_values(ascending=False).head(6)
recommended_movies = list(similar_user_rating.index)
recommended_movies.pop(0)
print "Recommended movies are : "
i=0
for a in recommended_movies:
    print a + "  " +str(similar_user_rating[i])
    i=i+1

similar_user

Recommended movies are : 
Modern Family  5.0
How to Train Your Dragon: The Hidden World  5.0
The Blacklist  5.0
Gotham  5.0
The Girl in the Spider's Web  5.0


878

In [162]:
movies_data['Name'].nunique()

193

## Matrix Decomposition


In [183]:
matrix_df = user_rating.pivot_table(index = 'UserID',columns ='MovieID', values = 'Rated',aggfunc='first').astype(float).fillna(0)
R = matrix_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
matrix_de_mean = R - user_ratings_mean.reshape(-1, 1)
U, sigma, Vt = svds(matrix_de_mean, k = 50)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
prediction_df = pd.DataFrame(all_user_predicted_ratings, columns = user_movie_rating.columns)
def recommend_movies(predictions_df, userID, movies_df, users_ratings_df, num_recommendations=5):

    user_number = userID - 1
    sorted_predictions = predictions_df.iloc[user_number].sort_values(ascending=False)

    user_data = users_ratings_df[users_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                             sort_values(['Rated'], ascending=False))

    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
                 merge(pd.DataFrame(sorted_predictions).reset_index(), how = 'left').
                 rename(columns = {user_number: 'Predictions'}).
                 sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1])
    return user_full, recommendations

already_rated, predictions = recommend_movies(prediction_df, 1000, movies_data,user_rating, 20)
recommended_movies = list(predictions['Name'].head(5))
recommended_movies

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


['Law & Order: Special Victims Unit',
 'A Star Is Born',
 'Frozen 2',
 'Riverdale',
 "Ocean's Eight"]